## Iterative K-means algorithm

In the previous lesson, we built our first version of a K-means-based algorithm. This initial version was capable of splitting the dataset twice, and we were able to notice some differences and improvements from the first to second split.

We'll now build from what we've done in order to develop a more robust, complete version of K-means. In this version, the code will cluster the data not twice, but as many times as necessary for the centroids to converge towards the mean of their clusters — or until it reaches a maximum number of iterations (N) that we'll set.

![](https://s3.amazonaws.com/dq-content/742/1.1-m742.svg)

Another important improvement from the first version is that we'll be able to use a different number of clusters (K) through the same code without being limited to splitting the dataset between only two clusters, as in lesson one.

Finally, in the new implementation, we'll still only work with two variables; however, we'll write more flexible code that does not rely on the variables' names, allowing the users to use any combinations of features they want to.

To achieve all this, we'll first need to initialize as many centroids as we want. Remember that in the previous lesson, our fetch_coordinates function was only capable of dealing with two centroids.

We'll use the Annual Income instead of the Age column, alongside Spending Score.

### Instructions
1. Read the mall_customers.csv file into a DataFrame called customers and keep the columns Annual Income and Spending Score.

2. Create a function called get_centroids that will receive two arguments: a DataFrame and K (the number of centroids to initialize).

- Inside the function, create a K-sized sample of the DataFrame.

- The function should return, in order:

    - The k-sized sample DataFrame
    - The data in the DataFrame as a list of lists in the same order
    
3. Call the function passing the customers DataFrame and k=2. Assign the result back to centroids and coords and print the outputs.

In [5]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns

In [ ]:
cols_to_keep = ['Annual Income', 'Spending Score']

customers = pd.read_csv("mall_customers.csv") 

customers = customers[cols_to_keep].copy()

def get_centroids(df, k):
    
    centroids = df.sample(k)
    
    coords = centroids.values.tolist()
    
    return centroids, coords

centroids, coords = get_centroids(customers, k = 2)

print(centroids, coords)

     Annual Income  Spending Score
182             98              15
90              59              55 [[98, 15], [59, 55]]


## Calculating Distances
You may remember how we calculated the distances between the centroids and the data points in the previous lesson. We used the Euclidean distance formula to create new columns in the customers DataFrame for each centroid. This means that we ended up with two columns: dist_centroid_1 and dist_centroid_2.

As we now need to calculate the distance for any number of centroids we want to, that approach won't work anymore. We need to write code that will create k new columns from the dist_centroid_1 to dist_centroid_k.

![](https://s3.amazonaws.com/dq-content/742/2.1-m742.svg)

In the code displayed on the right, we started creating a function called calculate_distance that takes a two-column DataFrame and the list of lists with the centroids' coordinates as inputs. Inside the function, we'll initialize an empty list called names. The function then loops through the centroids' coordinates list using enumerate, so we can have access to the value and the index of each inner list.

### Instructions
The variables customers, centroids, clusters, coords, and the get_centroids function are available in this screen.

1. Write a function, calculate_distance, to calculate the distance to each centroid.

    - It should take two arguments:
        - A two-column DataFrame
        - A list of coordinates of the centroids returned by the get_centroids function.

    - Loop through the centroids_coords list and create a column, dist_centroid_<suffix>, containing the distance of each row to that particular centroid.
        - In the column name, <suffix> should be appropriately replaced with a number between 1 and the number of clusters/centroids.

    - The function should return the DataFrame containing the new columns and a list with the name of those new columns.

2. Call the function using the customers DataFrame and the coords list as arguments and assign the results to customers and dist_names.

3. Print the outputs.

In [6]:
def calculate_distance(df, centroids_coords):
    
    names = []
    
    for i, centroid in enumerate(centroids_coords):
        name = f'dist_centroid_{i + 1}'
        df[name] = np.sqrt((df.iloc[:,0] - centroid[0])**2 + (df.iloc[:,1] - centroid[1])**2)
        names.append(name)
         
    return df, names

customers, dist_names = calculate_distance(customers, coords)

print(customers, dist_names)

     Annual Income  Spending Score  dist_centroid_1  dist_centroid_2
0               15              39        86.400231        46.818800
1               15              81       106.042444        51.107729
2               16               6        82.492424        65.192024
3               16              77       102.800778        48.301139
4               17              40        84.770278        44.598206
..             ...             ...              ...              ...
195            120              79        67.675697        65.551506
196            126              28        30.870698        72.235725
197            126              74        65.306967        69.641941
198            137              18        39.115214        86.330759
199            137              83        78.390050        82.873397

[200 rows x 4 columns] ['dist_centroid_1', 'dist_centroid_2']


## Assigning Clusters
We're now able to initialize as many centroids as we want and also calculate the distance from each point to as many centroids, too. The next step is to assign clusters.

When we limit the algorithm to only two clusters, it's much easier to determine the cluster for a point. It's only a matter of comparing two distances. We now need to be able to compare as many distances as we want, which demands a different approach.

That's the reason the calculate_distance function from the last screen returns not only the DataFrame with the distance columns, but also the names of such columns.

This list with names will be used to select only the columns containing the distances so we can select the one closer to the data point.

The DataFrame.idxmin() method returns the index of the minimum value in columns. However, if we set axis=1, it returns the columns with the minimum value in a row. We can then create the cluster column to select the column with the minimum distance:

```python
customers['cluster'] = customers[dist_names].idxmin(axis=1)
```

```
Annual Income	Spending Score	dist_centroid_1	dist_centroid_2	cluster
15	39	33.24154	23.345235	dist_centroid_2
15	81	43.139309	51.429563	dist_centroid_1
16	6	54.083269	36.400549	dist_centroid_2
16	77	39.698866	47.413078	dist_centroid_1
17	40	31.016125	21.587033	dist_centroid_2
```

### Instructions
In this exercise, we'll create a cluster column scaffolded from the above like this:

```
Annual Income	Spending Score	dist_centroid_1	dist_centroid_2	cluster
15	39	33.24154	23.345235	2
15	81	43.139309	51.429563	1
16	6	54.083269	36.400549	2
16	77	39.698866	47.413078	1
17	40	31.016125	21.587033	2
```

1. Create a column, `cluster`, that contains the cluster for each row as exemplified above.

2. Use the `sns.scatterplot` function to create a chart with the scatter plot of the `customers` DataFrame, grouped by cluster. In the same chart, plot the centroids.

In [7]:
customers['cluster'] = customers[dist_names].idxmin(axis=1).str.split('_').str[-1]
print(customers)

sns.scatterplot('Annual Income', 'Spending Score', hue='cluster', palette='tab10', data=customers, s=50)
sns.scatterplot('Annual Income', 'Spending Score', color='black', data=centroids, s=100)

plt.show()

     Annual Income  Spending Score  dist_centroid_1  dist_centroid_2 cluster
0               15              39        86.400231        46.818800       2
1               15              81       106.042444        51.107729       2
2               16               6        82.492424        65.192024       2
3               16              77       102.800778        48.301139       2
4               17              40        84.770278        44.598206       2
..             ...             ...              ...              ...     ...
195            120              79        67.675697        65.551506       2
196            126              28        30.870698        72.235725       1
197            126              74        65.306967        69.641941       1
198            137              18        39.115214        86.330759       1
199            137              83        78.390050        82.873397       1

[200 rows x 5 columns]


TypeError: scatterplot() got multiple values for argument 'data'

## Recalculating Centroids
It's now time to recalculate the centroids again, just like in this image from the previous lesson:


As we've seen before, the centroids are only randomly initialized once, and, from the second time on, they are the mean of all points in their cluster. This means that we do not need to use the get_centroids function again.

The process of recalculating centroids is similar to what we've done in the last lesson, but now we'll also extract the coordinates of the centroids.

### Instructions
We've created a list containing the two variables we're using for clustering.

1. Use the `DataFrame.groupby()` method to group these columns by the cluster column and calculate their mean.Assign the result back to `new_centroids`. Round the results to 4 digits.

2. Create a list of lists containing the coordinates for each centroid and assign it to new_coords.

3. Print the new centroids and the new coordinates.

In [ ]:
variables = customers.columns[:2]

new_centroids = round(customers.groupby('cluster')[variables].mean(), 4)
new_coords = new_centroids.values.tolist()

print(new_centroids)
print(new_coords)

Creating an Iterative Process
Learn
Until now, we've taken every step to build the algorithm separately. It's time to start putting the pieces together.

The process we've done so far consists of:

Initializing random centroids.
Calculating distances.
Assigning clusters.
Recalculating new centroids.
Recalculating distances.
Assigning new clusters.
This list consists of only one iteration of the K-means algorithm. However, because it's an iterative algorithm, we should make our code repeat part of the process over and over again until the clusters no longer change or a maximum number of iterations is reached. To achieve that, all the other steps aside from the initialization of the first centroids should be inside a for loop. The number of iterations in the for loop will be the maximum number of iterations performed by the algorithm.

In this screen, we'll start the entire process from the beginning, building on everything we've done so far.

For this screen's exercise, we have already

Read the DataFrame.

Selected the columns we want to use.

Created a list containing these columns.

Initialized the first centroids.



Instructions
Use a for loop to recreate the iteration 100 times. Inside the loop you will:

Use the calculate_distance function to create the columns containing the distance from each centroid in the customers DataFrame. As a reminder:
The function takes the DataFrame and the list of coordinates of the centroids as arguments.
The function returns the DataFrame containing the new columns and a list with the name of those new columns.
Assign a cluster to each row using idxmin().
Recalculate the centroids using groupby() and their coordinates.
The new centroids and coordinates must be assigned to the same variable as in the first initialization, and they should have the same datatypes.
When the loop is over, print the total number of iterations and make sure the algorithm ran 100 times.

Create the scatter plot of the customers grouped by clusters, and also plot the centroids. See how the clusters are split after 100 iterations.

In [ ]:
customers = pd.read_csv('mall_customers.csv')

cols_to_keep = ['Annual Income', 'Spending Score']
customers = customers[cols_to_keep].copy()

variables = customers.columns

centroids, coords = get_centroids(customers, 2)


for i in range(100): 
    customers, dist_names = calculate_distance(customers, coords)

    customers['cluster'] = customers[dist_names].idxmin(axis=1).str.split('_').str[-1]

    centroids = round(customers.groupby('cluster')[variables].mean(), 4)
    coords = centroids.values.tolist()

print(f'Total Iterations: {i + 1}')
                    
sns.scatterplot('Annual Income', 'Spending Score', hue='cluster', palette='tab10', data=customers, s=50)
sns.scatterplot('Annual Income', 'Spending Score', color='black', data=centroids, s=100)

plt.show()

On the previous screen, we saw the algorithm ran for one hundred times before defining a final cluster split. Just like the K number of clusters, the total number of iterations is a parameter that the user is capable of setting when using K-means.

Using the number of iterations as a parameter is very important in K-means, because depending on the first (and random) initialization of clusters, the algorithm may take a long time to stop.

This number of iterations is better known as the maximum number of iterations because the algorithm also has another condition to stop running and will not necessarily need to run all the iterations.

The other condition is when the centroids don't change from one iteration to another. When centroids don't change, it means that they are already the mean of their cluster and, therefore, won't change again, which is the condition to stop the iterations.

This condition is the final piece that is missing in our algorithm, and we'll implement it on this screen.

## Instructions
The entire code from the previous screen is written for us already, so we can just add the new implementations.

1. Inside the loop, create a copy of the coords variable before using the calculate_distance function and assign it to another variable called last_coords. These will be the coordinates of the centroids created in the previous iteration of the loop.

2. At the end of the loop, but still inside it, just after the centroids and coordinates are recalculated, check if they are equal to the last iteration's centroids, which are stored in last_cords.

3. If coords is equal to last_coords, stop the iteration.

#### Questions
1. Did the number of iterations change or the scatter plots change?

    - Neither the plot nor the number of iterations changed.

    - Both the plot and the number of iterations changed.

    - ==Only the number of iterations changed.==

    - Only the scatter plots changed.


2. If the number of iterations changed and the scatter plots are the same, how many fewer iterations were necessary to achieve the same result?

    - The number of iterations did not change.

    - 90

    - ==92==

    - 100

In [ ]:
customers = pd.read_csv('mall_customers.csv')

cols_to_keep = ['Annual Income', 'Spending Score']
customers = customers[cols_to_keep].copy()

variables = customers.columns

centroids, coords  = get_centroids(customers, 2)

for i in range(100): 
    customers, dist_names = calculate_distance(customers, coords)

    customers['cluster'] = customers[dist_names].idxmin(axis=1).str.split('_').str[-1]

    centroids = round(customers.groupby('cluster')[variables].mean(), 4)
    coords = centroids.values.tolist()

print(f'Total Iterations: {i + 1}')
                    
sns.scatterplot('Annual Income', 'Spending Score', hue='cluster', palette='tab10', data=customers, s=50)
sns.scatterplot('Annual Income', 'Spending Score', color='black', data=centroids, s=100)

plt.show()


customers = pd.read_csv('mall_customers.csv')

cols_to_keep = ['Annual Income', 'Spending Score']
customers = customers[cols_to_keep].copy()

variables = customers.columns

centroids, coords  = get_centroids(customers, 2)

for i in range(100):
    last_coords = coords.copy()

    customers, dist_names = calculate_distance(customers, coords)

    customers['cluster'] = customers[dist_names].idxmin(axis=1).str.split('_').str[-1]

    centroids = round(customers.groupby('cluster')[variables].mean(), 4)
    coords = centroids.values.tolist()
                    
    if last_coords == coords:
        break

print(f'Total Iterations: {i + 1}')
                    
sns.scatterplot('Annual Income', 'Spending Score', hue='cluster', palette='tab10', data=customers, s=50)
sns.scatterplot('Annual Income', 'Spending Score', color='black', data=centroids, s=100)

plt.show()

## Finishing the Algorithm

We've come a long way from understanding the concepts of unsupervised machine learning and K-means, to building our first version of the algorithm.

On the last screen, the number of iterations needed to converge was eight, as opposed to one hundred iterations on the screen before that. This means that the last 92 iterations did absolutely nothing and were a waste of time and computational power.

Now, our algorithm runs only when it needs to. If the algorithm does not converge or takes too long, it's important to set a maximum number of iterations so it does not run indefinitely.

We have gone through all the steps:

![](https://s3.amazonaws.com/dq-content/742/7.1-m742.svg)

On this screen, we'll combine these steps into a single function that we'll call our k-means algorithm. This function will:

1. Split the clusters.
2. Print the number of iterations needed.
3. Plot the final clusters and centroids.
4. Work for a two-column DataFrame and any number of clusters.
5. Allow us to set the maximum number of iterations.


### Instructions
1. Write a function called kmeans with the following properties:

    - The function should take a two-column DataFrame, the number of clusters k, and the maximum number of iterations (set the default to 100) as inputs.

    - The function should:

        - Create a list called variables, containing the columns used in the clusterization.
        - Randomly initialize centroids and their coordinates with the get_centroids function.
        - Create a loop that will run for the maximum number of iterations, or until the centroids no longer change.
            - Recalculate the centroids and their coordinates inside the loop.
        - Print the total number of iterations needed and create scatter plots for the data and for the centroids.
    - Return the cluster column.
    
2. Call the function using the customers DataFrame and k=2. Assign the result to clusters.

In [ ]:
customers = pd.read_csv('mall_customers.csv')

cols_to_keep = ['Annual Income', 'Spending Score']
customers = customers[cols_to_keep].copy()


def kmeans(df, k, n_iterations=100):
    variables = df.columns

    centroids, coords = get_centroids(df, k)

    for i in range(n_iterations):
        last_coords = coords.copy()

        df, dists = calculate_distance(df, coords)

        df['cluster'] = df[dists].idxmin(axis=1).str.split('_').str[-1]

        centroids = round(df.groupby('cluster')[variables].mean(), 4)
        coords = centroids.values.tolist()

        if last_coords == coords:
      	    break
    
    print(f'Total Iterations: {i + 1}')

    fig, ax = plt.subplots(figsize=(10, 5))
    sns.scatterplot(variables[0], variables[1], hue='cluster', palette='tab10', data=df, s=50, ax=ax)
    sns.scatterplot(variables[0], variables[1], color='black', data=centroids, s=100, ax=ax)


    plt.tight_layout()
    plt.show()

    return df['cluster']

  
clusters = kmeans(customers, 2)

## Review
In this lesson, we created a more complete version of the K-means algorithm. We're now able to split the dataset into clusters, independent of the variables' names. It's also possible to set the number of clusters and maximum number of iterations.

In the next lesson, we'll look into metrics and defining the number of clusters.